In [1]:
import numpy as np

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [2]:
class Module(object):
    """
    Basically, you can think of a module as of a something (black box)
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`:

        output = module.forward(input)

    The module should be able to perform a backward pass: to differentiate the `forward` function.
    More, it should be able to differentiate it if is a part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule.

        gradInput = module.backward(input, gradOutput)
    """
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True

    def forward(self, input):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        return self.updateOutput(input)

    def backward(self,input, gradOutput):
        """
        Performs a backpropagation step through the module, with respect to the given input.

        This includes
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput


    def updateOutput(self, input):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.

        Make sure to both store the data in `output` field and return it.
        """

        # The easiest case:

        # self.output = input
        # return self.output

        pass

    def updateGradInput(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own input.
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.

        The shape of `gradInput` is always the same as the shape of `input`.

        Make sure to both store the gradients in `gradInput` field and return it.
        """

        # The easiest case:

        # self.gradInput = gradOutput
        # return self.gradInput

        pass

    def accGradParameters(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass

    def zeroGradParameters(self):
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass

    def getParameters(self):
        """
        Returns a list with its parameters.
        If the module does not have parameters return empty list.
        """
        return []

    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters.
        If the module does not have parameters return empty list.
        """
        return []

    def train(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = True

    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = False

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want
        to have readable description.
        """
        return "Module"

# Sequential container

**Define** a forward and backward pass procedures.

In [3]:
class Sequential(Module):
    """
    This class implements a container, which processes `input` data sequentially.

    `input` is processed by each module (layer) in self.modules consecutively.
    The resulting array is called `output`.
    """

    def __init__(self):
        super(Sequential, self).__init__()
        self.modules = []

    def add(self, module):
        """
        Adds a module to the container.
        """
        self.modules.append(module)

    def updateOutput(self, input):
        """
        Basic workflow of FORWARD PASS:

            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})

        Just write a little loop.
        """
        self.output = input
        for module in self.modules:
            self.output = module.forward(self.output)
        return self.output

    def backward(self, input, gradOutput):
        """
        Workflow of BACKWARD PASS:

            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)
            gradInput = module[0].backward(input, g_1)

        !!!

        To each module you need to provide the input, module saw while forward pass,
        it is used while computing gradients.
        Make sure that the input for `i-th` layer the output of `module[i]` (just the same input as in forward pass)
        and NOT `input` to this Sequential module.

        !!!
        """
        # First we need to compute all forward passes to get intermediate outputs
        # We'll store them in a list
        inputs = [input]
        for module in self.modules[:-1]:
            inputs.append(module.forward(inputs[-1]))
        
        # Now do backward pass in reverse order
        self.gradInput = gradOutput
        for i in range(len(self.modules)-1, 0, -1):
            self.gradInput = self.modules[i].backward(inputs[i], self.gradInput)
        
        # Final backward pass for first module
        self.gradInput = self.modules[0].backward(inputs[0], self.gradInput)
        return self.gradInput

    def zeroGradParameters(self):
        for module in self.modules:
            module.zeroGradParameters()

    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]

    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]

    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string

    def __getitem__(self,x):
        return self.modules.__getitem__(x)

    def train(self):
        """
        Propagates training parameter through all modules
        """
        self.training = True
        for module in self.modules:
            module.train()

    def evaluate(self):
        """
        Propagates training parameter through all modules
        """
        self.training = False
        for module in self.modules:
            module.evaluate()

# Layers

## 1 (0.2). Linear transform layer
Also known as dense layer, fully-connected layer, FC-layer, InnerProductLayer (in caffe), affine transform
- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

In [5]:
import numpy as np

class Linear(Module):
    """
    A module which applies a linear transformation
    A common name is fully-connected layer, InnerProductLayer in caffe.

    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        self.n_in = n_in
        self.n_out = n_out
        
        # веса
        stdv = 1.0 / np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size=(n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size=n_out)
        
        # градиенты
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)
        
        # входы и выходы для слоя
        self.input = None
        self.output = None

    def updateOutput(self, input):
        self.input = input
        self.output = input @ self.W.T + self.b
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput @ self.W
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        self.gradW += gradOutput.T @ input
        self.gradb += gradOutput.sum(axis=0)

    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)

    def getParameters(self):
        return [self.W, self.b]

    def getGradParameters(self):
        return [self.gradW, self.gradb]

    def __repr__(self):
        return f"Linear({self.n_in} -> {self.n_out})"

## 2. (0.2) SoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{softmax}(x)_i = \frac{\exp x_i} {\sum_j \exp x_j}$

Recall that $\text{softmax}(x) == \text{softmax}(x - \text{const})$. It makes possible to avoid computing exp() from large argument.

In [7]:
class SoftMax(Module):
    def __init__(self):
         super(SoftMax, self).__init__()

    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        exp = np.exp(self.output)
        self.output = exp / exp.sum(axis=1, keepdims=True)
        return self.output

    def updateGradInput(self, input, gradOutput):
        s = self.output
        self.gradInput = s * (gradOutput - (s * gradOutput).sum(axis=1, keepdims=True))
        return self.gradInput

    def __repr__(self):
        return "SoftMax"

## 3. (0.2) LogSoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{logsoftmax}(x)_i = \log\text{softmax}(x)_i = x_i - \log {\sum_j \exp x_j}$

The main goal of this layer is to be used in computation of log-likelihood loss.

In [11]:
class LogSoftMax(Module):
    def __init__(self):
         super(LogSoftMax, self).__init__()

    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        #как будто так устойчивее чем через деление на экспоненту в конце
        logsum = np.log(np.exp(self.output).sum(axis=1, keepdims=True))
        self.output -= logsum
        return self.output

    def updateGradInput(self, input, gradOutput):
        s = np.exp(self.output)
        self.gradInput = gradOutput - s * gradOutput.sum(axis=1, keepdims=True)
        return self.gradInput

    def __repr__(self):
        return "LogSoftMax"

## 4. (0.3) Batch normalization
One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement the first part of the layer: features normalization. The second part (`ChannelwiseScaling` layer) is implemented below.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

The layer should work as follows. While training (`self.training == True`) it transforms input as $$y = \frac{x - \mu}  {\sqrt{\sigma + \epsilon}}$$
where $\mu$ and $\sigma$ - mean and variance of feature values in **batch** and $\epsilon$ is just a small number for numericall stability. Also during training, layer should maintain exponential moving average values for mean and variance:
```
    self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)
    self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)
```
During testing (`self.training == False`) the layer normalizes input using moving_mean and moving_variance.

Note that decomposition of batch normalization on normalization itself and channelwise scaling here is just a common **implementation** choice. In general "batch normalization" always assumes normalization + scaling.

In [12]:
class BatchNormalization(Module):
    EPS = 1e-3
    def __init__(self, alpha = 0.):
        super(BatchNormalization, self).__init__()
        self.alpha = alpha
        self.moving_mean = None
        self.moving_variance = None

    def updateOutput(self, input):
        if self.training:
            batch_mean = input.mean(axis=0)
            batch_var = input.var(axis=0)
            
            if self.moving_mean is None:  # first run
                self.moving_mean = batch_mean
                self.moving_variance = batch_var
            else:
                self.moving_mean = self.alpha * self.moving_mean + (1 - self.alpha) * batch_mean
                self.moving_variance = self.alpha * self.moving_variance + (1 - self.alpha) * batch_var
            
            self.output = (input - batch_mean) / np.sqrt(batch_var + self.EPS)
        else:
            self.output = (input - self.moving_mean) / np.sqrt(self.moving_variance + self.EPS)
            
        return self.output

    def updateGradInput(self, input, gradOutput):
        # как будто реально слишком наворочино, но мне проще в одну
        if self.training:
            batch_var = input.var(axis=0)
            n = input.shape[0]
            self.gradInput = (n * gradOutput - gradOutput.sum(axis=0) - 
                           (input - input.mean(axis=0)) * (gradOutput * (input - input.mean(axis=0))).sum(axis=0) / (batch_var + self.EPS)
            ) / (n * np.sqrt(batch_var + self.EPS))
        else:
            self.gradInput = gradOutput / np.sqrt(self.moving_variance + self.EPS)
            
        return self.gradInput

    def __repr__(self):
        return "BatchNormalization"

In [13]:
class ChannelwiseScaling(Module):
    """
    Implements linear transform of input y = γ * x + β
    where γ, β - learnable vectors of length x.shape[-1]
    """
    def __init__(self, n_out):
        super(ChannelwiseScaling, self).__init__()

        stdv = 1./np.sqrt(n_out)
        self.gamma = np.random.uniform(-stdv, stdv, size=n_out)
        self.beta = np.random.uniform(-stdv, stdv, size=n_out)

        self.gradGamma = np.zeros_like(self.gamma)
        self.gradBeta = np.zeros_like(self.beta)

    def updateOutput(self, input):
        self.output = input * self.gamma + self.beta
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * self.gamma
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        self.gradBeta = np.sum(gradOutput, axis=0)
        self.gradGamma = np.sum(gradOutput * input, axis=0)

    def zeroGradParameters(self):
        self.gradGamma.fill(0)
        self.gradBeta.fill(0)

    def getParameters(self):
        return [self.gamma, self.beta]

    def getGradParameters(self):
        return [self.gradGamma, self.gradBeta]

    def __repr__(self):
        return "ChannelwiseScaling"

Practical notes. If BatchNormalization is placed after a linear transformation layer (including dense layer, convolutions, channelwise scaling) that implements function like `y = weight * x + bias`, than bias adding become useless and could be omitted since its effect will be discarded while batch mean subtraction. If BatchNormalization (followed by `ChannelwiseScaling`) is placed before a layer that propagates scale (including ReLU, LeakyReLU) followed by any linear transformation layer than parameter `gamma` in `ChannelwiseScaling` could be freezed since it could be absorbed into the linear transformation layer.

## 5. (0.3) Dropout
Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. Here $p$ is probability of an element to be zeroed.

This has proven to be an effective technique for regularization and preventing the co-adaptation of neurons.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch), zero out elements and multiply elements by $1 / (1 - p)$. The latter is needed for keeping mean values of features close to mean values which will be in test mode. When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [15]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        self.p = p
        self.mask = None

    def updateOutput(self, input):
        if self.training:
            self.mask = (np.random.rand(*input.shape) > self.p) / (1 - self.p)
            self.output = input * self.mask
        else:
            self.output = input
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * self.mask if self.training else gradOutput
        return self.gradInput

    def __repr__(self):
        return f"Dropout(p={self.p})"

## 6. (2.0) Conv2d
Implement [**Conv2d**](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html). Use only this list of parameters: (in_channels, out_channels, kernel_size, stride, padding, bias, padding_mode) and fix dilation=1 and groups=1.

In [16]:
class Conv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=0, bias=True, padding_mode='zeros'):
        super(Conv2d, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size
        self.stride = (stride, stride) if isinstance(stride, int) else stride
        self.padding = (padding, padding) if isinstance(padding, int) else padding
        self.bias = bias
        self.padding_mode = padding_mode

        scale = 1 / np.sqrt(in_channels * self.kernel_size[0] * self.kernel_size[1])
        self.weight = np.random.uniform(-scale, scale, 
                                     (out_channels, in_channels, *self.kernel_size))
        self.gradWeight = np.zeros_like(self.weight)
        self.bias = np.random.uniform(-scale, scale, out_channels) if bias else None
        self.gradBias = np.zeros_like(self.bias) if bias else None

    def updateOutput(self, input):
        input_padded = np.pad(input, 
                            [(0,0), (0,0), 
                             (self.padding[0], self.padding[0]), 
                             (self.padding[1], self.padding[1])], 
                            'constant') if any(self.padding) else input
        
        batch_size, _, in_h, in_w = input_padded.shape
        out_h = (in_h - self.kernel_size[0]) // self.stride[0] + 1
        out_w = (in_w - self.kernel_size[1]) // self.stride[1] + 1
        
        self.output = np.zeros((batch_size, self.out_channels, out_h, out_w))
        
        for i in range(out_h):
            for j in range(out_w):
                h_start = i * self.stride[0]
                w_start = j * self.stride[1]
                window = input_padded[:, :, 
                                   h_start:h_start+self.kernel_size[0], 
                                   w_start:w_start+self.kernel_size[1]]
                self.output[:, :, i, j] = np.tensordot(window, self.weight, 
                                                     axes=([1,2,3], [1,2,3]))
        
        self.output += self.bias[None, :, None, None] if self.bias is not None else 0
        return self.output

    def updateGradInput(self, input, gradOutput):
        input_padded = np.pad(input,
                            [(0,0), (0,0),
                             (self.padding[0], self.padding[0]),
                             (self.padding[1], self.padding[1])],
                            'constant') if any(self.padding) else input
        
        batch_size, _, in_h, in_w = input_padded.shape
        self.gradInput = np.zeros_like(input_padded)
        
        for i in range(gradOutput.shape[2]):
            for j in range(gradOutput.shape[3]):
                h_start = i * self.stride[0]
                w_start = j * self.stride[1]
                self.gradInput[:, :,
                             h_start:h_start+self.kernel_size[0],
                             w_start:w_start+self.kernel_size[1]] += \
                    np.tensordot(gradOutput[:, :, i, j], self.weight, axes=([1],[0]))
        
        self.gradInput = self.gradInput[:, :,
                                      self.padding[0]:in_h-self.padding[0],
                                      self.padding[1]:in_w-self.padding[1]] if any(self.padding) else self.gradInput
        return self.gradInput

    def __repr__(self):
        return f"Conv2d(in={self.in_channels}, out={self.out_channels}, kernel={self.kernel_size}, stride={self.stride}, pad={self.padding})"

## 7. (0.5) Implement [**MaxPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) and [**AvgPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html). 
Use only parameters like kernel_size, stride, padding (negative infinity for maxpool and zero for avgpool) and other parameters fixed as in framework.

In [17]:
class MaxPool2d(Module):
    def __init__(self, kernel_size, stride, padding):
        super(MaxPool2d, self).__init__()
        self.kernel_size = kernel_size if isinstance(kernel_size, tuple) else (kernel_size, kernel_size)
        self.stride = stride if isinstance(stride, tuple) else (stride, stride)
        self.padding = padding if isinstance(padding, tuple) else (padding, padding)

    def updateOutput(self, input):
        # Добавляем padding с -inf для maxpool
        pad_h, pad_w = self.padding
        if pad_h > 0 or pad_w > 0:
            input_padded = np.pad(input, 
                                [(0,0), (0,0), 
                                 (pad_h, pad_h), 
                                 (pad_w, pad_w)], 
                                mode='constant', 
                                constant_values=-np.inf)
        else:
            input_padded = input

        batch, channels, in_h, in_w = input_padded.shape
        k_h, k_w = self.kernel_size
        s_h, s_w = self.stride
        
        # Вычисляем размеры выхода
        out_h = (in_h - k_h) // s_h + 1
        out_w = (in_w - k_w) // s_w + 1
        
        self.output = np.zeros((batch, channels, out_h, out_w))
        
        # Применяем max pooling
        for i in range(out_h):
            for j in range(out_w):
                h_start = i * s_h
                w_start = j * s_w
                window = input_padded[:, :, 
                                   h_start:h_start+k_h, 
                                   w_start:w_start+k_w]
                self.output[:, :, i, j] = np.max(window, axis=(2,3))
                
        return self.output

    def updateGradInput(self, input, gradOutput):
        pad_h, pad_w = self.padding
        if pad_h > 0 or pad_w > 0:
            input_padded = np.pad(input,
                                [(0,0), (0,0),
                                 (pad_h, pad_h),
                                 (pad_w, pad_w)],
                                mode='constant',
                                constant_values=-np.inf)
        else:
            input_padded = input
            
        batch, channels, in_h, in_w = input_padded.shape
        k_h, k_w = self.kernel_size
        s_h, s_w = self.stride
        out_h, out_w = gradOutput.shape[2], gradOutput.shape[3]
        
        self.gradInput = np.zeros_like(input_padded)
        
        # Распространяем градиент только на максимальные элементы
        for i in range(out_h):
            for j in range(out_w):
                h_start = i * s_h
                w_start = j * s_w
                window = input_padded[:, :, 
                                    h_start:h_start+k_h, 
                                    w_start:w_start+k_w]
                max_mask = (window == np.max(window, axis=(2,3), keepdims=True))
                self.gradInput[:, :, 
                             h_start:h_start+k_h, 
                             w_start:w_start+k_w] += \
                    gradOutput[:, :, i, j][:, :, None, None] * max_mask
        
        # Убираем padding если был
        if pad_h > 0 or pad_w > 0:
            self.gradInput = self.gradInput[:, :, 
                                          pad_h:-pad_h, 
                                          pad_w:-pad_w]
            
        return self.gradInput

    def __repr__(self):
        return f"MaxPool2d(kernel={self.kernel_size}, stride={self.stride}, pad={self.padding})"


class AvgPool2d(Module):
    def __init__(self, kernel_size, stride, padding):
        super(AvgPool2d, self).__init__()
        self.kernel_size = kernel_size if isinstance(kernel_size, tuple) else (kernel_size, kernel_size)
        self.stride = stride if isinstance(stride, tuple) else (stride, stride)
        self.padding = padding if isinstance(padding, tuple) else (padding, padding)

    def updateOutput(self, input):
        # Добавляем padding с 0 для avgpool
        pad_h, pad_w = self.padding
        if pad_h > 0 or pad_w > 0:
            input_padded = np.pad(input, 
                                [(0,0), (0,0), 
                                 (pad_h, pad_h), 
                                 (pad_w, pad_w)], 
                                mode='constant')
        else:
            input_padded = input

        batch, channels, in_h, in_w = input_padded.shape
        k_h, k_w = self.kernel_size
        s_h, s_w = self.stride
        
        # Вычисляем размеры выхода
        out_h = (in_h - k_h) // s_h + 1
        out_w = (in_w - k_w) // s_w + 1
        
        self.output = np.zeros((batch, channels, out_h, out_w))
        
        # Применяем average pooling
        for i in range(out_h):
            for j in range(out_w):
                h_start = i * s_h
                w_start = j * s_w
                window = input_padded[:, :, 
                                   h_start:h_start+k_h, 
                                   w_start:w_start+k_w]
                self.output[:, :, i, j] = np.mean(window, axis=(2,3))
                
        return self.output

    def updateGradInput(self, input, gradOutput):
        pad_h, pad_w = self.padding
        if pad_h > 0 or pad_w > 0:
            input_padded = np.pad(input,
                                [(0,0), (0,0),
                                 (pad_h, pad_h),
                                 (pad_w, pad_w)],
                                mode='constant')
        else:
            input_padded = input
            
        batch, channels, in_h, in_w = input_padded.shape
        k_h, k_w = self.kernel_size
        s_h, s_w = self.stride
        out_h, out_w = gradOutput.shape[2], gradOutput.shape[3]
        
        self.gradInput = np.zeros_like(input_padded)
        
        # Распределяем градиент равномерно по всем элементам окна
        norm_factor = 1.0 / (k_h * k_w)
        for i in range(out_h):
            for j in range(out_w):
                h_start = i * s_h
                w_start = j * s_w
                self.gradInput[:, :, 
                             h_start:h_start+k_h, 
                             w_start:w_start+k_w] += \
                    gradOutput[:, :, i, j][:, :, None, None] * norm_factor
        
        # Убираем padding если был
        if pad_h > 0 or pad_w > 0:
            self.gradInput = self.gradInput[:, :, 
                                          pad_h:-pad_h, 
                                          pad_w:-pad_w]
            
        return self.gradInput

    def __repr__(self):
        return f"AvgPool2d(kernel={self.kernel_size}, stride={self.stride}, pad={self.padding})"

## 8. (0.3) Implement **GlobalMaxPool2d** and **GlobalAvgPool2d**. 
They do not have testing and parameters are up to you but they must aggregate information within channels. Write test functions for these layers on your own.

In [19]:
class GlobalMaxPool2d(Module):
    def __init__(self):
        super(GlobalMaxPool2d, self).__init__()
    
    def updateOutput(self, input):
        # Схлопываем пространственные измерения (H,W), оставляя каналы
        self.output = np.max(input, axis=(2, 3))
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        batch, channels, h, w = input.shape
        self.gradInput = np.zeros_like(input)
        
        # Находим позиции максимальных элементов
        max_indices = np.argmax(input.reshape(batch, channels, -1), axis=2)
        
        # Распределяем градиенты только на максимальные элементы
        for b in range(batch):
            for c in range(channels):
                pos = np.unravel_index(max_indices[b,c], (h,w))
                self.gradInput[b, c, pos[0], pos[1]] = gradOutput[b,c]
        
        return self.gradInput
    
    def __repr__(self):
        return "GlobalMaxPool2d()"


class GlobalAvgPool2d(Module):
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()
    
    def updateOutput(self, input):
        # Усредняем по пространственным измерениям (H,W)
        self.output = np.mean(input, axis=(2, 3))
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        batch, channels, h, w = input.shape
        norm_factor = 1.0 / (h * w)
        
        # Равномерно распределяем градиент по всем элементам
        self.gradInput = np.ones_like(input) * gradOutput[:, :, None, None] * norm_factor
        return self.gradInput
    
    def __repr__(self):
        return "GlobalAvgPool2d()"


# Тесты
def test_global_pools():
    np.random.seed(42)
    
    # Тестовый тензор (2 примера, 3 канала, 4x4)
    x = np.random.rand(2, 3, 4, 4)
    
    # Тестируем GlobalMaxPool2d
    max_pool = GlobalMaxPool2d()
    out_max = max_pool.updateOutput(x)
    print("GlobalMaxPool output shape:", out_max.shape)  # Должно быть (2, 3)
    
    # Проверяем правильность вычисления максимумов
    for b in range(2):
        for c in range(3):
            assert np.allclose(out_max[b,c], np.max(x[b,c]))
    
    # Тестируем градиенты
    grad_out = np.ones_like(out_max)
    grad_in = max_pool.updateGradInput(x, grad_out)
    print("GlobalMaxPool grad_input shape:", grad_in.shape)  # Должно совпадать с входом (2,3,4,4)
    
    # Тестируем GlobalAvgPool2d
    avg_pool = GlobalAvgPool2d()
    out_avg = avg_pool.updateOutput(x)
    print("GlobalAvgPool output shape:", out_avg.shape)  # Должно быть (2, 3)
    
    # Проверяем правильность вычисления средних
    for b in range(2):
        for c in range(3):
            assert np.allclose(out_avg[b,c], np.mean(x[b,c]))
    
    # Тестируем градиенты
    grad_in_avg = avg_pool.updateGradInput(x, grad_out)
    print("GlobalAvgPool grad_input shape:", grad_in_avg.shape)  # Должно совпадать с входом
    
    print("Ок")

# Запускаем тесты
test_global_pools()

GlobalMaxPool output shape: (2, 3)
GlobalMaxPool grad_input shape: (2, 3, 4, 4)
GlobalAvgPool output shape: (2, 3)
GlobalAvgPool grad_input shape: (2, 3, 4, 4)
Ок


## 9. (0.2) Implement 
[**Flatten**](https://pytorch.org/docs/stable/generated/torch.flatten.html)

In [21]:
class Flatten(Module):
    def __init__(self, start_dim=0, end_dim=-1):
        super(Flatten, self).__init__()
        self.start_dim = start_dim
        self.end_dim = end_dim
        self.input_shape = None  # Для сохранения формы входа

    def updateOutput(self, input):
        # Сохраняем исходную форму
        self.input_shape = input.shape
        
        # Обрабатываем отрицательные индексы
        end_dim = self.end_dim if self.end_dim >= 0 else len(self.input_shape) + self.end_dim
        
        # Вычисляем новую форму
        new_shape = (
            *self.input_shape[:self.start_dim],
            -1,  # Схлопываем все размерности между start_dim и end_dim
            *self.input_shape[end_dim+1:]
        )
        
        self.output = input.reshape(new_shape)
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Восстанавливаем исходную форму
        self.gradInput = gradOutput.reshape(self.input_shape)
        return self.gradInput

    def __repr__(self):
        return f"Flatten(start_dim={self.start_dim}, end_dim={self.end_dim})"

# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**):

In [22]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()

    def updateOutput(self, input):
        self.output = np.maximum(input, 0)
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput

    def __repr__(self):
        return "ReLU"

## 10. (0.1) Leaky ReLU
Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope.

In [23]:
class LeakyReLU(Module):
    def __init__(self, slope=0.03):
        super(LeakyReLU, self).__init__()
        self.slope = slope

    def updateOutput(self, input):
        self.output = np.where(input > 0, input, input * self.slope)
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * np.where(input > 0, 1, self.slope)
        return self.gradInput

    def __repr__(self):
        return f"LeakyReLU(slope={self.slope})"

## 11. (0.1) ELU
Implement [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activations.

In [24]:
class ELU(Module):
    def __init__(self, alpha=1.0):
        super(ELU, self).__init__()
        self.alpha = alpha

    def updateOutput(self, input):
        self.output = np.where(input > 0, input, self.alpha * (np.exp(input) - 1))
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * np.where(input > 0, 1, self.output + self.alpha)
        return self.gradInput

    def __repr__(self):
        return f"ELU(alpha={self.alpha})"

## 12. (0.1) SoftPlus
Implement [**SoftPlus**](https://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29) activations. Look, how they look a lot like ReLU.

In [ ]:
class SoftPlus(Module):
    def __init__(self):
        super(SoftPlus, self).__init__()

    def updateOutput(self, input):
        # Your code goes here. ################################################
        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        return self.gradInput

    def __repr__(self):
        return "SoftPlus"

## 13. (0.2) Gelu
Implement [**Gelu**](https://pytorch.org/docs/stable/generated/torch.nn.GELU.html) activations.

In [25]:
class SoftPlus(Module):
    def __init__(self):
        super(SoftPlus, self).__init__()

    def updateOutput(self, input):
        self.output = np.log1p(np.exp(input))  # Более устойчиво 
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * (1 - 1/(1 + np.exp(input)))  # Сигмоида
        return self.gradInput

    def __repr__(self):
        return "SoftPlus"

# Criterions

Criterions are used to score the models answers.

In [26]:
class Criterion(object):
    def __init__(self):
        self.output = None
        self.gradInput = None

    def forward(self, input, target):
        return self.updateOutput(input, target)

    def backward(self, input, target):
        return self.updateGradInput(input, target)

    def updateOutput(self, input, target):
        self.output = None  # loss
        return self.output

    def updateGradInput(self, input, target):
        self.gradInput = None 
        return self.gradInput

    def __repr__(self):
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.
- input:   **`batch_size x n_feats`**
- target: **`batch_size x n_feats`**
- output: **scalar**

In [27]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()

    def updateOutput(self, input, target):
        self.output = np.mean((input - target)**2)  # mSE
        return self.output

    def updateGradInput(self, input, target):
        # Your code goes here. ################################################
        self.gradInput = 2 * (input - target) / input.shape[0]  # Градиент MSE
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

## 14. (0.2) Negative LogLikelihood criterion (numerically unstable)
You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula,
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. Also there is a small hack with adding small number to probabilities to avoid computing log(0).
- input:   **`batch_size x n_feats`** - probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**



In [28]:
class ClassNLLCriterionUnstable(Criterion):
    EPS = 1e-15
    def __init__(self):
        super(ClassNLLCriterionUnstable, self).__init__()

    def updateOutput(self, input, target):
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        
        self.output = -np.sum(target * np.log(input_clamp)) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        
        self.gradInput = -target / (input_clamp * input.shape[0])
        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterionUnstable"

## 15. (0.3) Negative LogLikelihood criterion (numerically stable)
- input:   **`batch_size x n_feats`** - log probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**

Task is similar to the previous one, but now the criterion input is the output of log-softmax layer. This decomposition allows us to avoid problems with computation of forward and backward of log().

In [29]:
class ClassNLLCriterion(Criterion):
    def __init__(self):
        super(ClassNLLCriterion, self).__init__()

    def updateOutput(self, input, target):
        self.output = -np.mean(np.sum(target * input, axis=1))
        return self.output

    def updateGradInput(self, input, target):
        self.gradInput = -target / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterion"

1-я часть задания: реализация слоев, лосей и функций активации - 5 баллов. \\
2-я часть задания: реализация моделей на своих классах. Что должно быть:
  1. Выберите оптимизатор и реализуйте его, чтоб он работал с вами классами. - 1 балл.
  2. Модель для задачи мультирегрессии на выбраных вами данных. Использовать FCNN, dropout, batchnorm, MSE. Пробуйте различные фукнции активации. Для первой модели попробуйте большую, среднюю и маленькую модель. - 1 балл.
  3. Модель для задачи мультиклассификации на MNIST. Использовать свёртки, макспулы, флэттэны, софтмаксы - 1 балла.
  4. Автоэнкодер для выбранных вами данных. Должен быть на свёртках и полносвязных слоях, дропаутах, батчнормах и тд. - 2 балла. \\

Дополнительно в оценке каждой модели будет учитываться:
1. Наличие правильно выбранной метрики и лосс функции.
2. Отрисовка графиков лосей и метрик на трейне-валидации. Проверка качества модели на тесте.
3. Наличие шедулера для lr.
4. Наличие вормапа.
5. Наличие механизма ранней остановки и сохранение лучшей модели.
6. Свитч лося (метрики) и оптимайзера.